<a href="https://colab.research.google.com/github/Oleg007003/A-model-for-predicting-the-outcome-of-a-football-game/blob/main/ConstructModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

План действий:

1)Вписать каждому игроку свои личное характеристики +

1.1)Исследовать дивизион

2)Наклепать модели

3)Наклепать дополнительные признаки, улучшающие модель

4)Научить выборку дообучаться online

5)Сохранить модели

# Блок с загрузкой данных

In [ ]:
!pip install catboost

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import catboost as cb
import keras
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import normalize
import math
from pandas.plotting import table 

In [ ]:
data = pd.read_csv('http://assets.codeforces.com/rounds/1522/3056cab52876d3327edafe4771c8589a96cb1a93/train.csv')
del data[data.columns[0]]

In [ ]:
data.head(10)

,Division,Time,home_team,away_team,full_time_home_goals,full_time_away_goals,half_time_home_goals,half_time_away_goals,Referee,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_corners,away_corners,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_coef,draw_coef,away_coef
0,0,NaN,152.0,426.0,0.0,0.0,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.627690,3.577721,2.198600
1,0,NaN,216.0,341.0,2.0,2.0,0.0,1.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.608321,4.251999,6.533805
2,0,NaN,223.0,119.0,2.0,2.0,2.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166811,3.486596,3.821467
3,0,NaN,68.0,137.0,2.0,1.0,0.0,1.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.259244,3.592401,3.460103
4,1,NaN,425.0,225.0,2.0,2.0,1.0,0.0,-1,13.0,11.0,5.0,7.0,18.0,15.0,4.0,10.0,3.0,4.0,1.0,0.0,2.424278,3.414191,3.282876
5,1,NaN,393.0,300.0,0.0,0.0,0.0,0.0,-1,6.0,6.0,1.0,2.0,9.0,13.0,6.0,8.0,1.0,0.0,0.0,0.0,2.787163,3.375001,2.817568
6,1,NaN,239.0,140.0,0.0,0.0,0.0,0.0,-1,16.0,4.0,8.0,1.0,9.0,16.0,6.0,1.0,0.0,2.0,0.0,0.0,2.080239,3.582070,3.998117
7,1,NaN,211.0,424.0,2.0,0.0,1.0,0.0,-1,18.0,5.0,7.0,3.0,12.0,10.0,6.0,6.0,2.0,1.0,0.0,1.0,2.099892,3.449822,4.099789
8,1,NaN,99.0,5.0,1.0,0.0,0.0,0.0,-1,9.0,5.0,5.0,2.0,5.0,9.0,6.0,4.0,0.0,1.0,0.0,0.0,1.312957,5.807312,13.129574
9,0,NaN,303.0,382.0,1.0,0.0,1.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.528475,4.476952,7.553624


In [ ]:
data.shape

(39992, 24)

Для обработки пропусков заполним их просто средними значениями

In [ ]:
for i in range(9, 21):
  name = data.columns[i]
  data[name] = data[name].fillna(data[name].mean())

Добавим признаков с результатом игры

In [ ]:
data['is_draw'] = (data['full_time_home_goals'] == data['full_time_away_goals']) + 0
data['is_win'] = (data['full_time_home_goals'] > data['full_time_away_goals']) + 0
data['is_lose'] = (data['full_time_home_goals'] < data['full_time_away_goals']) + 0
data['result'] = data['is_win'] - data['is_lose']

In [ ]:
#data = data[data['Division'] == 0]
data.shape

(1789, 28)

In [ ]:
X = data.drop(['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals'], axis = 1)
y = data[['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals']]
Priznak = data.copy()
del Priznak['result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state= 13, shuffle = False)
Priznak_train, Priznak_test = train_test_split(Priznak, test_size = 0.15, random_state= 13, shuffle = False)

In [ ]:
del Priznak_train['Referee']

In [ ]:
comands_home = data['home_team'].unique()
comands_away = data['away_team'].unique()
print(comands_home, comands_away, sep='\n')
  #print(row[['home_team', 'away_team', 'result']])

[152. 216. 223.  68. 303.  75. 138. 399. 232. 164. 119.   4. 426. 341.
 455. 177. 382. 137. 200. 176.  63. 235. 373. 323. 213. 129. 178. 435.
 209. 486. 219.]
[426. 341. 119. 137. 382. 164.   4. 177. 455. 399. 152. 232. 223.  68.
  75. 216. 138. 303.  63. 235. 200. 176. 213. 373. 129. 323. 178. 209.
 486. 435. 219.]


Посмотрим на корреляцию наших признаков с итогами игры

In [ ]:
tX = X.copy()
tX['ans'] = y['result']
cr = tX.corr()

In [ ]:
cr

,Division,Time,home_team,away_team,Referee,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_corners,away_corners,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_coef,draw_coef,away_coef,ans
Division,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_team,NaN,NaN,1.000000,-0.044211,NaN,0.003960,-0.050751,-0.003646,-0.046110,-0.001865,0.039086,0.018569,-0.020305,-0.040273,0.020986,0.000476,0.064229,-0.029079,0.047567,0.045907,0.011094
away_team,NaN,NaN,-0.044211,1.000000,NaN,-0.025658,-0.009323,-0.001442,0.011476,-0.032299,0.009874,0.003157,-0.007818,-0.014067,-0.026472,0.034065,-0.001611,0.017500,0.007918,-0.017430,0.007009
Referee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_shots,NaN,NaN,0.003960,-0.025658,NaN,1.000000,-0.186162,0.522008,-0.092636,0.058785,0.073336,0.447689,-0.290025,0.012835,0.032184,-0.166487,0.109907,-0.064338,-0.001819,0.024443,-0.017375
away_shots,NaN,NaN,-0.050751,-0.009323,NaN,-0.186162,1.000000,-0.092145,0.526122,0.175967,0.059102,-0.261693,0.369248,0.146645,0.082736,0.124282,-0.048459,0.048558,-0.045782,-0.070484,-0.025709
home_shots_on_target,NaN,NaN,-0.003646,-0.001442,NaN,0.522008,-0.092145,1.000000,-0.090840,-0.076375,-0.062656,0.225270,-0.085074,-0.095600,-0.078380,-0.154251,0.042482,-0.050349,0.032423,0.040587,0.100984
away_shots_on_target,NaN,NaN,-0.046110,0.011476,NaN,-0.092636,0.526122,-0.090840,1.000000,0.141616,-0.066489,-0.087936,0.154835,0.148734,0.039229,0.170612,-0.018211,0.048074,-0.003602,-0.044046,-0.115776
home_fouls,NaN,NaN,-0.001865,-0.032299,NaN,0.058785,0.175967,-0.076375,0.141616,1.000000,0.296185,-0.093131,0.026791,0.383906,0.171575,0.039862,0.007555,0.027750,-0.046522,-0.046251,-0.006801


# Блок про личные характеристики

In [ ]:
name_home_priznak = ['home_team', 'Division', 'full_time_home_goals', 'half_time_home_goals', 'home_shots', 'home_shots_on_target', 'home_fouls', 'home_corners', 'home_yellow_cards', 'home_red_cards', 'home_coef']
name_away_priznak = ['away_team', 'full_time_away_goals', 'half_time_away_goals', 'away_shots', 'away_shots_on_target', 'away_fouls', 'away_corners', 'away_yellow_cards', 'away_red_cards', 'away_coef']
name_draw_priznak = ['draw_coef']

In [ ]:
home_priznak = Priznak_train[name_home_priznak]
away_priznak = Priznak_train[name_away_priznak]
draw_priznak = Priznak_train[name_draw_priznak]

Получим значения минимального и максимального игрока(считаем, что подавляющее большинство номеров команд заполнено). Получим, что индексация начинается с 0. Тогда можно за 0ю команду считать команду без индекса

In [ ]:
minPlayerId = min(min(data['home_team']), min(data['away_team'])) # == 1
maxPlayerId = max(max(data['home_team']), max(data['away_team'])) # == 494

players = pd.DataFrame()
players['id'] = range(0, int(maxPlayerId) + 1)
lst_home = []
lst_away = []
for i in range(0, int(maxPlayerId) + 1):
  lst_home.append(Priznak_train[Priznak_train['home_team'] == i].shape[0])
  lst_away.append(Priznak_train[Priznak_train['away_team'] == i].shape[0])
players['games_home'] = pd.DataFrame(lst_home)
players['games_away'] = pd.DataFrame(lst_away)

In [ ]:
players

,id,games_home,games_away
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,50,51
...,...,...,...
482,482,0,0
483,483,0,0
484,484,0,0
485,485,0,0


In [ ]:
maxPlayerId

486.0

Теперь мы должны поставить нашим игрокам пропуски

In [ ]:
p_home = pd.DataFrame()
for cols in range(1, len(name_home_priznak)):
  name = name_home_priznak[cols]
  lst = [0]
  for i in range(int(minPlayerId), int(maxPlayerId) + 1):
    lst.append(home_priznak[home_priznak['home_team'] == i][name].sum())
  p_home['players_'+name] = lst

In [ ]:
p_away = pd.DataFrame()
for cols in range(1, len(name_away_priznak)):
  name = name_away_priznak[cols]
  lst = [0]
  for i in range(int(minPlayerId), int(maxPlayerId) + 1):
    lst.append(away_priznak[away_priznak['away_team'] == i][name].sum())
  p_away['players_'+name] = lst

In [ ]:
players = players.drop(0)
p_home = p_home.drop(0)
p_away = p_away.drop(0)

In [ ]:
p_home

,players_Division,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef
1,0,66.0,22.0,644.364292,244.856136,607.051336,287.455939,83.764397,4.268793,139.517610
2,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
479,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
480,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
481,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
482,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
p_home[p_home.columns[0]] //= players['games_home']
for i in range(1, len(p_home.columns)):
  p_home[p_home.columns[i]] /= players['games_home']

In [ ]:
for i in range(0, len(p_away.columns)):
  p_away[p_away.columns[i]] /= players['games_away']

In [ ]:
p_away.reset_index(drop=True, inplace=True)
p_home.reset_index(drop=True, inplace=True)
players.reset_index(drop=True, inplace=True)
a = pd.concat([p_away, p_home], axis = 1)
a.reset_index(drop=True, inplace=True)
players = pd.concat([players, a], axis = 1)

In [ ]:
'''players = players[players['games_home'] + players['games_away'] > 0]
for i in range(3, 21):
  name = players.columns[i]
  players[name] = players[name].fillna(players[name].mean())'''

"players = players[players['games_home'] + players['games_away'] > 0]\nfor i in range(3, 21):\n  name = players.columns[i]\n  players[name] = players[name].fillna(players[name].mean())"

In [ ]:
players

,id,games_home,games_away,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,players_Division,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef
0,1,0,0,inf,inf,inf,inf,inf,inf,inf,inf,inf,NaN,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,50,51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,482,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,483,0,0,inf,inf,inf,inf,inf,inf,inf,inf,inf,NaN,inf,inf,inf,inf,inf,inf,inf,inf,inf
483,484,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,485,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Теперь делим на среднее

In [ ]:
copy_player = players.copy()
del copy_player['id']

In [ ]:
copy_player.columns

Index(['games_home', 'games_away', 'players_full_time_away_goals',
       'players_half_time_away_goals', 'players_away_shots',
       'players_away_shots_on_target', 'players_away_fouls',
       'players_away_corners', 'players_away_yellow_cards',
       'players_away_red_cards', 'players_away_coef', 'players_Division',
       'players_full_time_home_goals', 'players_half_time_home_goals',
       'players_home_shots', 'players_home_shots_on_target',
       'players_home_fouls', 'players_home_corners',
       'players_home_yellow_cards', 'players_home_red_cards',
       'players_home_coef'],
      dtype='object')

Добавим наши новые признаки в базе:

In [ ]:
names_home = ['players_full_time_home_goals', 'players_half_time_home_goals',
       'players_home_shots', 'players_home_shots_on_target',
       'players_home_fouls', 'players_home_corners',
       'players_home_yellow_cards', 'players_home_red_cards',
       'players_home_coef']
names_away = ['players_full_time_away_goals',
       'players_half_time_away_goals', 'players_away_shots',
       'players_away_shots_on_target', 'players_away_fouls',
       'players_away_corners', 'players_away_yellow_cards',
       'players_away_red_cards', 'players_away_coef']

In [ ]:
df_h = pd.DataFrame()
df_a = pd.DataFrame()
df_h[names_home] = np.nan
df_a[names_away] = np.nan

In [ ]:
for index, row in Priznak_train.iterrows():
  home = (players[players['id'] == int(row['home_team'])][names_home])
  away = (players[players['id'] == int(row['away_team'])][names_away])
  df_h = df_h.append(home, ignore_index=True)
  df_a = df_a.append(away, ignore_index=True)

# Блок про дележку на блоки:

In [ ]:
class Division:
  def __init__(self, full_data, number):
    self.number = number
    self.data = full_data[full_data['Division'] == number].copy()
    self.data.reset_index(drop=True, inplace=True)
    
    self.X = self.data.drop(['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals'], axis = 1)
    self.y = self.data[['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals']]
    self.Priznak = self.data.copy()
    del self.Priznak['result']
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.15, random_state= 13, shuffle = False)
    self.Priznak_train, self.Priznak_test = train_test_split(self.Priznak, test_size = 0.15, random_state= 13, shuffle = False)

    del self.Priznak_train['Referee']

    

# Блок с моделями:

In [ ]:
X_real_train = X_train[['Division', 'Time', 'home_team', 'away_team', 'Referee', 'home_coef', 'draw_coef', 'away_coef']]
X_real_train.reset_index(drop=True, inplace=True)
a = pd.concat([df_h, df_a], axis = 1)
a.reset_index(drop=True, inplace=True)
X_real_train = pd.concat([X_real_train, a], axis = 1)

In [ ]:
#X_real_train['home_coef_log'] = np.log((X_real_train['home_coef'] - 1) / 10)
#X_real_train['home_delta_away'] = (X_real_train['home_coef'] - X_real_train['away_coef']) ** 3
#X_real_train['home_delta_draw'] = (X_real_train['home_coef'] - X_real_train['draw_coef']) ** 3
#X_real_train['draw_delta_away'] = (X_real_train['draw_coef'] - X_real_train['away_coef']) ** 3
#X_real_train['ones'] = X_real_train['home_coef'] ** 0
#X_real_train['draw_coef_log'] = np.log((X_real_train['draw_coef'] - 1) / 10)
#X_real_train['away_coef_log'] = np.log((X_real_train['away_coef'] - 1) / 10)
#X_real_train['player_home_shots_square'] = X_real_train['player_home_shots_on_target'] ** 2
#X_real_train['player_away_shots_square'] = X_real_train['player_away_shots_on_target'] ** 2

In [ ]:
X_real_train

In [ ]:
normalized_X=(X_real_train-X_real_train.mean())/X_real_train.std()
tX = normalized_X.copy()
tX['ans'] = y_train['result']
cr = tX.corr()
cr

In [ ]:
cat = cb.CatBoostClassifier()
grid = {'learning_rate': [0.03, 0.05, 0.6, 0.075, 0.875, 0.1, 0.11, 0.125, 0.5, 0.8, 1],
        'depth': [3, 4, 5, 6, 7, 8, 9, 10],
        'l2_leaf_reg': [0.1, 0.5, 1, 1.6, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.5, 4, 5, 7, 9, 11],
        'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
        'auto_class_weights': ['None', 'Balanced', 'SqrtBalanced'],
        'od_pval': [0, 0.0000000001, 0.0000001, 0.00001, 0.0001, 0.001, 0.01],
        'posterior_sampling': [True, False]}
cat.randomized_search(grid, X_real_train, y_train['result'], n_iter = 3, cv = 4)
#cat = cat.fit(normalized_X,y_train['result'], verbose=False)

Теперь проделаем аналогичные действия с Тренеровочными данными

In [ ]:
df_test_h = pd.DataFrame()
df_test_a = pd.DataFrame()
df_test_h[names_home] = np.nan
df_test_a[names_away] = np.nan

In [ ]:
lst_player_home_shots = []
for index, row in Priznak_test.iterrows():
  home = (players[players['id'] == int(row['home_team'])][names_home])
  away = (players[players['id'] == int(row['away_team'])][names_away])
  df_test_h = df_test_h.append(home, ignore_index=True)
  df_test_a = df_test_a.append(away, ignore_index=True)

In [ ]:
X_real_test = X_test[['Division', 'Time', 'home_team', 'away_team', 'Referee', 'home_coef', 'draw_coef', 'away_coef',]]
a = pd.concat([df_test_h, df_test_a], axis = 1)
X_real_test.reset_index(drop=True, inplace=True)
a.reset_index(drop=True, inplace=True)
X_real_test = pd.concat([X_real_test, a], axis = 1)

In [ ]:
normalized_X_test = (X_real_test-X_real_train.mean())/X_real_train.std()

In [ ]:
print(cat.score(X_real_train,y_train['result']))

In [ ]:
predict = cat.predict_proba(X_real_test)
real_result = y_test[['is_lose', 'is_draw', 'is_win']].to_numpy()
coefs = X_real_test[['away_coef', 'draw_coef', 'home_coef']].to_numpy()

In [ ]:
def is_put(ver, coeff):
  return ver * (coeff - 1) - (1 - ver) > 0.45 and ver > 0.44

In [ ]:
mx = -10000
porogmx = 0
for veroy in range(400, 801):
  right = 0
  coins = 0
  all = 0
  porog = veroy / 1000
  for i in range(predict.shape[0]):
    if predict[i][0] > porog:
      all += 1
      coins -= 1
      if real_result[i][0] == 1:
        right += 1
        coins += coefs[i][0]
    elif predict[i][1] > porog:
      coins -= 1
      all += 1
      if real_result[i][1] == 1:
        right += 1
        coins += coefs[i][1]
    elif predict[i][2] > porog:
      coins -= 1
      all += 1
      if real_result[i][2] == 1:
        right += 1
        coins += coefs[i][2]
  if coins > mx:
    mx = coins
    porogmx = porog
print(mx, porogmx)

In [ ]:
X_real_test.shape

(269, 26)